In [ ]:
# imports and setup
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse
from PIL import Image

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
def rescale_image(data):
    pdata_tmp = data.copy()
    m = np.nanmean(pdata_tmp)
    vplmin = m/2.
    vpmin = np.log10(vplmin)
    vpmax = np.log10(m * 100.)
    pdata_tmp[pdata_tmp<vplmin] = vplmin
    pdata_tmp = np.log10(pdata_tmp)
    return pdata_tmp,vpmin,vpmax

In [ ]:
# reading image into array
hdu = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
data = hdu[0].data

In [ ]:
# showing the image and saving it
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('original.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# measuring the image background
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

In [ ]:
# printing the "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluating the background as 2-d array
bkg_image = bkg.back()

In [ ]:
# showing the background and saving it
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('background.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# evaluating the background noise as 2-d array
bkg_rms = bkg.rms()

In [ ]:
# showing the background noise and saving it
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('backgroundnoise.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# subtracting the background from the data
data_sub = data - bkg

In [ ]:
# detecting amount of objects in the no background image
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
len(objects)

In [ ]:
# plotting background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plotting an ellipse that shows each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
# saving final version
plt.savefig('objectsshown.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# available fields
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# showing the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
# histogram the flux
n = len(flux)
width = 0.5
histmin = np.floor(min(flux))
histmax = np.ceil(max(flux)) + width

bins = np.arange(histmin, histmax, width)
plt.hist(flux, bins = bins, alpha = 0.5, edgecolor = "black")
plt.ylabel("fluxes per source")
plt.xlabel("sources")
plt.show

In [ ]:
mean = np.mean(flux)
std = np.std(flux)
bigoutlier = 0
print("the mean is " + str(mean))
print("the median is " + str(np.median(flux)))
print("the standard deviation of the fluxes is " + str(std))

if (abs(min(flux) - mean) > abs(max(flux - mean))):
    bigoutlier = min(flux)
else :
    bigoutlier = max(flux)

print("the biggest outlier is " + str(bigoutlier))

distance = (bigoutlier - mean) / std
print("the biggest outlier is " + str(distance) + " standard deviations away from the mean")

In [ ]:
hdu2 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")
data2 = hdu2[0].data

hdu3 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")
data3 = hdu3[0].data

In [ ]:
data_res, dmin, dmax = rescale_image(data)
data2_res, d2min, d2max = rescale_image(data2)
data3_res, d3min, d3max = rescale_image(data3)

In [ ]:
data_res[data_res<dmin] = dmin
data_res[data_res>dmax] = dmax
data2_res[data2_res<dmin] = d2min
data2_res[data2_res>dmax] = d2max
data3_res[data3_res<dmin] = d3min
data3_res[data3_res>dmax] = d3max

In [ ]:
rgb = np.zeros((data_res.shape[0], data2_res.shape[1], 3))
rgb[:,:,0] = (data_res - dmin) / (dmax - dmin)
rgb[:,:,1] = (data2_res - d2min) / (d2max - d2min)
rgb[:,:,2] = (data3_res - d3min) / (d3max - d3min)

In [ ]:
f,ax = plt.subplots(1,1,figsize =(20,20))
ax.axis('off')
ax.imshow(rgb)
plt.savefig('rgb.png', bbox_inches = "tight", dpi = 600)